# LangGraph Router (Without LLM)

A router decides which path to take based on conditions.
It's like a traffic controller deciding which route cars take.

## Install LangGraph

In [4]:
!pip install langgraph grandalf

from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

In [5]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal

## Define State

In [6]:
# Get OpenAI API Key from environment variables
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    masked_key = api_key[:10] + "..." + api_key[-5:] if len(api_key) > 15 else "***"
    print(f"✅ OpenAI API Key loaded: {masked_key}")
else:
    print("⚠️  OpenAI API Key not found in .env file")
    print("   Add your key to .env file: OPENAI_API_KEY=your_key_here")

NameError: name 'os' is not defined

## Check Environment Variables

In [ ]:
class RouteState(TypedDict):
    city: str
    temperature: int
    message: str

## City Information

In [ ]:
cities = {
    "Chennai": {"temperature": 35, "type": "hot"},
    "Ooty": {"temperature": 15, "type": "cold"},
    "Madurai": {"temperature": 32, "type": "hot"},
    "Kodaikanal": {"temperature": 18, "type": "cold"}
}

## Create Nodes

In [ ]:
def check_temperature(state: RouteState) -> RouteState:
    """
    This node gets the temperature of a city.
    """
    city = state["city"]
    temp = cities[city]["temperature"]
    
    return {
        "city": city,
        "temperature": temp,
        "message": f"{city} temperature: {temp}°C"
    }

def handle_hot_city(state: RouteState) -> RouteState:
    """
    This node handles hot cities.
    """
    return {
        "city": state["city"],
        "temperature": state["temperature"],
        "message": f"{state['city']} is hot! Drink water and stay cool! 💧"
    }

def handle_cold_city(state: RouteState) -> RouteState:
    """
    This node handles cold cities.
    """
    return {
        "city": state["city"],
        "temperature": state["temperature"],
        "message": f"{state['city']} is cold! Wear warm clothes! 🧥"
    }

## Create Router Function

The router decides which path to take based on temperature.

In [ ]:
def route_by_temperature(state: RouteState) -> Literal["hot", "cold"]:
    """
    This is the router!
    It looks at temperature and decides:
    - If temperature > 30, go to 'hot' path
    - If temperature <= 30, go to 'cold' path
    """
    if state["temperature"] > 30:
        return "hot"
    else:
        return "cold"

## Build the Graph with Router

In [ ]:
# Create graph
graph = StateGraph(RouteState)

# Add nodes
graph.add_node("check_temp", check_temperature)
graph.add_node("hot_handler", handle_hot_city)
graph.add_node("cold_handler", handle_cold_city)

# Add edges
graph.add_edge(START, "check_temp")

# Add conditional edges (ROUTER)
# After check_temp, the router decides which path to take
graph.add_conditional_edges(
    "check_temp",
    route_by_temperature,
    {
        "hot": "hot_handler",
        "cold": "cold_handler"
    }
)

# Both paths lead to END
graph.add_edge("hot_handler", END)
graph.add_edge("cold_handler", END)

# Compile
app = graph.compile()

print("✅ Graph with Router is ready!")

✅ Graph with Router is ready!


## Test the Router with Hot City

In [ ]:
# Test with Chennai (hot city)
initial_state = {
    "city": "Chennai",
    "temperature": 0,
    "message": ""
}

result = app.invoke(initial_state)

print("\n" + "="*50)
print(f"City: {result['city']}")
print(f"Temperature: {result['temperature']}°C")
print(f"Action: {result['message']}")
print("="*50)


City: Chennai
Temperature: 35°C
Action: Chennai is hot! Drink water and stay cool! 💧


## Test the Router with Cold City

In [ ]:
# Test with Ooty (cold city)
initial_state = {
    "city": "Ooty",
    "temperature": 0,
    "message": ""
}

result = app.invoke(initial_state)

print("\n" + "="*50)
print(f"City: {result['city']}")
print(f"Temperature: {result['temperature']}°C")
print(f"Action: {result['message']}")
print("="*50)


City: Ooty
Temperature: 15°C
Action: Ooty is cold! Wear warm clothes! 🧥


## Test Multiple Cities

In [ ]:
test_cities = ["Chennai", "Ooty", "Madurai", "Kodaikanal"]

print("\nTesting all cities:")
print("="*50)

for city in test_cities:
    initial_state = {
        "city": city,
        "temperature": 0,
        "message": ""
    }
    
    result = app.invoke(initial_state)
    
    print(f"\n{city}:")
    print(f"  Temperature: {result['temperature']}°C")
    print(f"  Action: {result['message']}")

print("\n" + "="*50)


Testing all cities:

Chennai:
  Temperature: 35°C
  Action: Chennai is hot! Drink water and stay cool! 💧

Ooty:
  Temperature: 15°C
  Action: Ooty is cold! Wear warm clothes! 🧥

Madurai:
  Temperature: 32°C
  Action: Madurai is hot! Drink water and stay cool! 💧

Kodaikanal:
  Temperature: 18°C
  Action: Kodaikanal is cold! Wear warm clothes! 🧥



## Draw the Graph

In [ ]:
# Draw the graph structure
try:
    print("\nGraph Structure:")
    print(app.get_graph().draw_ascii())
except Exception as e:
    print(f"Graph drawing error: {e}")
    print("\nManual Graph Structure:")
    print("""
              ┌─────────────┐
              │    START    │
              └──────┬──────┘
                     │
                     ▼
            ┌────────────────┐
            │  check_temp    │
            └────────┬───────┘
                     │
        ┌────────────┴────────────┐
        │                         │
        │ (ROUTER)                │
        │ temp > 30?              │
        │                         │
        ▼                         ▼
   ┌─────────────┐          ┌──────────────┐
   │ hot_handler │          │ cold_handler │
   └──────┬──────┘          └────────┬─────┘
          │                          │
          └──────────┬───────────────┘
                     │
                     ▼
              ┌─────────────┐
              │     END     │
              └─────────────┘
    """)


Graph Structure:
               +-----------+                
               | __start__ |                
               +-----------+                
                     *                      
                     *                      
                     *                      
              +------------+                
              | check_temp |                
              +------------+                
              ..           ..               
            ..               ..             
          ..                   ..           
+--------------+           +-------------+  
| cold_handler |           | hot_handler |  
+--------------+           +-------------+  
              **           **               
                **       **                 
                  **   **                   
                +---------+                 
                | __end__ |                 
                +---------+                 


## How the Router Works

The router is the decision maker in your graph:

1. **check_temp** node gets the temperature
2. **Router function** looks at temperature and decides:
   - If temp > 30: Go to "hot_handler" path
   - If temp <= 30: Go to "cold_handler" path
3. **Handle the result** in the appropriate handler
4. **End** the workflow

### Key Code:
```python
def route_by_temperature(state: RouteState) -> Literal["hot", "cold"]:
    if state["temperature"] > 30:
        return "hot"
    else:
        return "cold"

graph.add_conditional_edges(
    "check_temp",
    route_by_temperature,
    {
        "hot": "hot_handler",
        "cold": "cold_handler"
    }
)
```

This is exactly how routers work - no LLM needed! 🚀